In [2]:
import json
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from google.colab import files

In [3]:
batch_size = 32
image_height = 224
image_width = 224

In [4]:
#Function to compile model using the Adam optimizer

def compile_model(model, loss="categorical_crossentropy", metrics=["accuracy"]):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    variables = model.trainable_variables
    optimizer.build(variables)
    return model.compile(loss=loss,
                         optimizer=optimizer,
                         metrics=metrics)

In [13]:
#Creating a custom CNN model with Conv2D, MaxPool2D, Dropout, Flatten and Dense layers

custom_cnn_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=10,
                           kernel_size=3,
                           activation="relu",
                           input_shape=(224,224,3)),
    tf.keras.layers.Conv2D(filters=10,
                           kernel_size=3,
                           activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2,
                              padding="valid"),
    tf.keras.layers.Conv2D(filters=10,
                           kernel_size=3,
                           activation="relu",
                           name="conv2d_3_activations_layer"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.MaxPool2D(pool_size=2,
                              padding="valid"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(67,activation="sigmoid")
])

In [6]:
compile_model(custom_cnn_model)

In [7]:
custom_cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 10)      280       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 10)      910       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 10)     0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 108, 108, 10)      910       
                                                                 
 dropout (Dropout)           (None, 108, 108, 10)      0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 10)       0         
 2D)                                                    

In [8]:
#Function to create a feature extraction layer for pre-trained models

def create_model(model_url, num_classes=67):
    feature_extractor_layer = hub.KerasLayer(model_url,
                                            trainable=False,
                                            name="feature_extraction_layer",
                                            input_shape=(image_width,image_height)+(3,))
    model = tf.keras.Sequential([
        feature_extractor_layer,
        layers.Dense(num_classes, activation="softmax", name="output_layer")])

    return model

In [9]:
#Storing the urls of EfficientNetB0, MobileNet_V2, ResNet_V1, Inception_V2

model_urls = {"EfficientNetB0" : "https://tfhub.dev/tensorflow/efficientnet/b0/classification/1",
              "MobileNet_V2" : "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4",
              "ResNet_V1" : "https://tfhub.dev/google/imagenet/resnet_v1_101/classification/5",
              "Inception_V2" : "https://tfhub.dev/google/imagenet/inception_v2/classification/5"}

In [10]:
#Creating a dictionary of model name and corresponding Sequential model object

models = {}
for name, url in model_urls.items():
    model_name = name + "_model"
    exec(f"{model_name} = create_model(url,67)") #this creates a model using the create_model function and a url from the model_urls dictionary
    exec(f"models[model_name]= {model_name}")

In [11]:
models

{'EfficientNetB0_model': <keras.engine.sequential.Sequential at 0x7ef8ac35a8f0>,
 'MobileNet_V2_model': <keras.engine.sequential.Sequential at 0x7ef7ffc01690>,
 'ResNet_V1_model': <keras.engine.sequential.Sequential at 0x7ef7ffbcca90>,
 'Inception_V2_model': <keras.engine.sequential.Sequential at 0x7ef7fdfa7070>}

In [12]:
#Compiles the 4 feature extraction models, prints their summaries and saves the model files

for name, model in models.items():
    compile_model(model)
    print(name)
    exec(f"{name}.summary()")
    print("\n %-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-% \n %-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-% \n \n")
    exec(f"{name}.save('/content/drive/MyDrive/IndoorSceneClassification/compiled_models/{name}.h5')")
    save_model_file_path = '/content/drive/MyDrive/IndoorSceneClassification/compiled_models/' + name + '.h5'
    #files.download(save_model_file_path)

EfficientNetB0_model
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (K  (None, 1000)             5330564   
 erasLayer)                                                      
                                                                 
 output_layer (Dense)        (None, 67)                67067     
                                                                 
Total params: 5,397,631
Trainable params: 67,067
Non-trainable params: 5,330,564
_________________________________________________________________

 %-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-% 
 %-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-%-% 
 

MobileNet_V2_model
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (K  (None